In [4]:
import os
from collections import Counter
import seaborn as sns
import matplotlib.pyplot as plt
import myServices as ms
import models as md
import numpy as np
import pandas as pd
import sklearn
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,roc_curve, auc, roc_auc_score, f1_score
import joblib

In [ ]:
# to compute ececution time do: 
# with timeit():
#     # your code, e.g., 
class timeit(): 
    from datetime import datetime
    def __enter__(self):
        self.tic = self.datetime.now()
    def __exit__(self, *args, **kwargs):
        print('runtime: {}'.format(self.datetime.now() - self.tic))

In [6]:
import pandas as pd
dataSetPath = 'datasets/datasetBasin1.csv'
basinDataSet = pd.read_csv(dataSetPath, index_col = None)
print(basinDataSet.isna().sum())
basinDataSet.describe()

percentage        0
disToRiv          0
TWI               1
TPI               0
FAcc              0
slope         14442
elevation         0
x_coord           0
y_coord           0
dtype: int64


,percentage,disToRiv,TWI,TPI,FAcc,slope,elevation,x_coord,y_coord
count,2.493980e+06,2.493980e+06,2.493979e+06,2.493980e+06,2.493980e+06,2.479538e+06,2.493980e+06,2.493980e+06,2.493980e+06
mean,9.150033e-02,2.097987e+02,9.478135e+00,1.100693e-04,4.330615e+04,4.821367e+00,1.455343e+02,3.650845e+05,5.264240e+06
std,6.592287e-01,1.727702e+02,4.079174e+00,1.950585e-01,1.029387e+06,4.994698e+00,4.576501e+01,3.175800e+03,1.830620e+03
min,0.000000e+00,0.000000e+00,-9.252120e+00,-7.461320e+00,2.500000e+01,4.700000e-04,-3.808000e-02,3.586730e+05,5.260213e+06
25%,0.000000e+00,7.424622e+01,5.551585e+00,-6.195000e-02,8.772391e+01,1.600760e+00,1.159088e+02,3.623130e+05,5.262718e+06
50%,0.000000e+00,1.691892e+02,1.025464e+01,-2.100000e-04,2.851745e+02,3.192175e+00,1.483494e+02,3.650780e+05,5.264348e+06
75%,0.000000e+00,3.030264e+02,1.288554e+01,6.131000e-02,1.254889e+03,6.192805e+00,1.800543e+02,3.676380e+05,5.265788e+06
max,5.000000e+00,1.355775e+03,2.486353e+01,6.706970e+00,6.282632e+07,7.002943e+01,2.605691e+02,3.721230e+05,5.267983e+06


## Importing and manipulating datasets

In [ ]:
sklearn.metrics.get_scorer_names()

In [7]:
### Cleaning basin1DataSet 
dataSetPath = 'datasets/datasetBasin1.csv'
basinDataSet = pd.read_csv(dataSetPath, index_col = None)
# basin1Light = pd.read_csv('datasetBasin1_NoDataFree.csv', index_col = None)
# print(basinDataSet.info())
# basinDataSet.describe()

In [ ]:
basinDataSet.isna().any()

In [ ]:
colNames = ['percentage','DLSOL5R200', 'DLSOL4R150', 'DLSOL5R150']
for col in colNames: 
    basinDataSet[col].fillna(0,inplace=True)

In [9]:
basinDataSet.dropna(subset=['slope'],inplace=True)

In [ ]:
basinDataSet.drop(['fid'], axis =1, inplace=True)

In [10]:
basinDataSet.isna().sum()

percentage    0
disToRiv      0
TWI           0
TPI           0
FAcc          0
slope         0
elevation     0
x_coord       0
y_coord       0
dtype: int64

In [ ]:
#### NOrmalize Flow Accumulation
basinDataSet['FAcc'] = (basinDataSet['FAcc']- basinDataSet['FAcc'].min())/(basinDataSet['FAcc'].max()-basinDataSet['FAcc'].min())


In [ ]:
### Replacing QGIS NoData value(-9999) with 0 
repalcer  = basinDataSet['FAProx_01'].to_numpy()
basinDataSet['FAProx_01'] = [0 if repalcer[j] == -9999 else repalcer[j] for j in range(len(repalcer))]                                                                                                                         
                                                                                                                          

In [11]:
## Transform a column datatype
repalcer  = basinDataSet['percentage'].to_numpy().astype('int')
basinDataSet.loc[:,'percentage'] = repalcer

In [12]:
basinDataSet.describe()

,percentage,disToRiv,TWI,TPI,FAcc,slope,elevation,x_coord,y_coord
count,2.479538e+06,2.479538e+06,2.479538e+06,2.479538e+06,2.479538e+06,2.479538e+06,2.479538e+06,2.479538e+06,2.479538e+06
mean,9.201795e-02,2.084064e+02,9.483515e+00,-3.328848e-04,4.352760e+04,4.821367e+00,1.453463e+02,3.650828e+05,5.264239e+06
std,6.610626e-01,1.713850e+02,4.081659e+00,1.949152e-01,1.031605e+06,4.994698e+00,4.569706e+01,3.169308e+03,1.824893e+03
min,0.000000e+00,0.000000e+00,-9.252120e+00,-7.461320e+00,2.500000e+01,4.700000e-04,-3.685000e-02,3.586780e+05,5.260218e+06
25%,0.000000e+00,7.382411e+01,5.553190e+00,-6.226000e-02,8.892947e+01,1.600760e+00,1.157656e+02,3.623130e+05,5.262723e+06
50%,0.000000e+00,1.677051e+02,1.025636e+01,-2.700000e-04,2.881975e+02,3.192175e+00,1.481632e+02,3.650730e+05,5.264348e+06
75%,0.000000e+00,3.010399e+02,1.289330e+01,6.091000e-02,1.265309e+03,6.192805e+00,1.797259e+02,3.676280e+05,5.265783e+06
max,5.000000e+00,1.349713e+03,2.486353e+01,6.706970e+00,6.282450e+07,7.002943e+01,2.605691e+02,3.721180e+05,5.267978e+06


In [13]:
basinDataSet.to_csv('datasets/basin1_FirstFeatureSet_Clean.csv', index=None)

In [ ]:
ds = DS.head(5)
s = {}
s['Datas'] = ds
print(s)

## Proportional Split 

In [14]:
## Stratified Split
from sklearn.model_selection import StratifiedShuffleSplit

X,Y = ms.importDataSet('datasets/basin1_FirstFeatureSet_Clean.csv', 'percentage')
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=50)
for train_index, test_index in sss.split(X, Y):
    print("TRAIN:", train_index.size, "TEST:", test_index.size)
    X_train = X.iloc[train_index]
    y_train = Y.iloc[train_index]
    X_test = X.iloc[test_index]
    y_test = Y.iloc[test_index]

TRAIN: 1983630 TEST: 495908


In [15]:
## Describing training set
print(len(X_train['elevation']), len(y_train) )
trainCount = Counter(y_train)
print(trainCount)

1983630 1983630
Counter({0: 1941324, 5: 35056, 1: 7250})


In [16]:
#####    Creating training set     #####
X_train.loc[:,'percentage'] = y_train
X_train.head()


/var/folders/s5/cqkwnnps0f1_35_bccj0y8q80000gn/T/ipykernel_58406/795959617.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train.loc[:,'percentage'] = y_train


,disToRiv,TWI,TPI,FAcc,slope,elevation,x_coord,y_coord,percentage
2419089,393.85910,10.20537,0.54161,27.04812,4.70770,152.60564,367648,5261043,0
2388753,102.59142,16.06245,-0.05203,9458.78809,1.66023,197.74588,362193,5261148,0
494166,110.00000,13.18089,0.00468,530.13470,1.49737,144.01004,361608,5266063,0
61008,136.01471,10.95948,0.01625,57.49655,8.96356,169.10132,362008,5267198,0
1203168,90.13878,2.90234,-0.04885,53.52814,4.69841,85.86313,366483,5264443,0


In [17]:
## Removing coordinates from training set
X_train.drop(['x_coord','y_coord'], axis =1, inplace=True)
X_train.head()

/var/folders/s5/cqkwnnps0f1_35_bccj0y8q80000gn/T/ipykernel_58406/1019043263.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train.drop(['x_coord','y_coord'], axis =1, inplace=True)


,disToRiv,TWI,TPI,FAcc,slope,elevation,percentage
2419089,393.85910,10.20537,0.54161,27.04812,4.70770,152.60564,0
2388753,102.59142,16.06245,-0.05203,9458.78809,1.66023,197.74588,0
494166,110.00000,13.18089,0.00468,530.13470,1.49737,144.01004,0
61008,136.01471,10.95948,0.01625,57.49655,8.96356,169.10132,0
1203168,90.13878,2.90234,-0.04885,53.52814,4.69841,85.86313,0


In [18]:
X_train.to_csv('datasets/basin1_FirstFeatureSet_Clean_Training.csv', index=None)

In [19]:
#####. Creating Test set
print(X_test.head())
X_test.loc[:,'percentage'] = y_test
print(X_test.head())
print(X_test.info())
testCount = Counter(X_test['percentage'])
print(f"testCount:  {testCount}")


          disToRiv       TWI      TPI        FAcc     slope  elevation  \
1149772   65.76473  15.88676  0.10242  7934.72021   5.60282  108.43757   
1110309  579.82758   9.04799  0.01117  1758.07971   1.66141  165.68309   
98254     65.19202   1.79303  0.54303    64.37885  13.80903  142.59601   
242062   215.92822  13.59605 -0.12878   802.94952   1.87882  166.73584   
1093319  166.17009   3.08946  0.01888   275.23352   5.40569  170.24129   

         x_coord  y_coord  
1149772   364493  5264573  
1110309   369238  5264668  
98254     371563  5267053  
242062    362363  5266648  
1093319   360698  5264703  
          disToRiv       TWI      TPI        FAcc     slope  elevation  \
1149772   65.76473  15.88676  0.10242  7934.72021   5.60282  108.43757   
1110309  579.82758   9.04799  0.01117  1758.07971   1.66141  165.68309   
98254     65.19202   1.79303  0.54303    64.37885  13.80903  142.59601   
242062   215.92822  13.59605 -0.12878   802.94952   1.87882  166.73584   
1093319  166.1700

/var/folders/s5/cqkwnnps0f1_35_bccj0y8q80000gn/T/ipykernel_58406/345470071.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test.loc[:,'percentage'] = y_test


In [20]:
X_test.to_csv('datasets/basin1_FirstFeatureSet_Clean_Test.csv', index=None)

In [21]:
## This proportions are the reason why a sample_weight of 0.01 for the majority class give best results for regression
totalTrain = sum([trainCount[0], trainCount[1], trainCount[5]]) 
totalValidation = sum([testCount[0], testCount[1], testCount[5]])
print(f"total Train samples: {totalTrain},  total Validation samples: {totalValidation}")
print("Summary of traning and test dataset class balance")
print(f"Training Set:", '\n', "Class 0: %.3f" %(trainCount[0]/totalTrain), " Class 1: %.4f" %(trainCount[1]/totalTrain), "Class 5: %.4f"%(trainCount[5]/totalTrain))
print("Testing Set:", '\n', "Class 0: %.3f" %(testCount[0]/totalValidation)," Class 1: %.4f" %(testCount[1]/totalValidation),  "Class 5: %.4f"%(testCount[5]/totalValidation))



total Train samples: 1983630,  total Validation samples: 495908
Summary of traning and test dataset class balance
Training Set: 
 Class 0: 0.979  Class 1: 0.0037 Class 5: 0.0177
Testing Set: 
 Class 0: 0.979  Class 1: 0.0037 Class 5: 0.0177


In [ ]:
model = ms.loadModel('./outputs/2022-08-05/00-35-58/2208050035.pkl')
dataSetToSave = ms.makePredictionToImportAsSHP(csvName, model, X, Y, 'percentage')
print(dataSetToSave.head())

## Combining dataSets to build AllVsOne_training and OneVsAll_test

In [8]:
# Concat datasets
sourceFileForDatasets = 'datasets/'
allDataSetsNames = ['basin1Light_Clean.csv', 'basin2_CleanDataSet.csv', 'basin3_CleanDataSet_copy.csv','basin4_CleanDataSet.csv','basin5_CleanDataSet.csv']
# OneVsAllDataSetName = 'basin1Light_Clean.csv'

for datasetForTest in allDataSetsNames:
    filename, file_extension = os.path.splitext(datasetForTest)
    newListOfNames = [s for s in allDataSetsNames if s != datasetForTest]
    allDataSetsFileName = 'allVs_'+ filename +'_Training'
    DFToConcatAll = pd.DataFrame()
#     DFToConcatAll = pd.read_csv((sourceFileForDatasets+datasetForTest), index_col = None)
#     print(DFToConcatAll.head())
    for datasets in newListOfNames:
        DFToConcatAll = pd.concat([DFToConcatAll, pd.read_csv((sourceFileForDatasets+datasets), index_col = None)])
    nameToSafe = sourceFileForDatasets+allDataSetsFileName+file_extension
    DFToConcatAll.drop(['x_coord','y_coord'], axis =1, inplace=True)
    DFToConcatAll.to_csv(nameToSafe, index=None)    
    



## Controled sampling

In [ ]:
DS = pd.read_csv('datasets/basin2 _Training.csv', index_col = None)
print(DS.head())


In [ ]:
print(DS.columns)

In [ ]:
plt.boxplot(DS['FAProx_01']) # , , DS['elevation'], DS['disToRiv']]

In [ ]:
## Resampling appliying class selection by rule:

# RULE1: Select point at a distance to river less than 300m. 

# # newDS = pseudoClassCreation(DS, "distanceToRiver", 300, 2)
def pseudoClassCreation(dataset, conditionVariable, threshold, pseudoClass, targetClassName):
    '''
    Replace <targetClass> by  <pseudoClass> where <conditionVariable >= threshold>. 
    Return:
      dataset with new classes group. 
    '''
    datsetReclassified = dataset.copy()
    actualTarget = (np.array(dataset[targetClassName])).ravel()
    conditionVar = (np.array(dataset[conditionVariable])).ravel()
    datsetReclassified[targetClassName] = [ pseudoClass if conditionVar[j] >= threshold 
                                           else actualTarget[j]
                                           for j in range(len(actualTarget))]
    print(Counter(datsetReclassified[targetClassName]))
    return  datsetReclassified

def revertPseudoClassCreation(dataset, originalClass, pseudoClass, targetClassName):
    '''
    Restablich  <targetClass> with <originalClass> where <targetClassName == pseudoClass>. 
    Return:
      dataset with original classes group. 
    '''
    datsetReclassified = dataset.copy()
    actualTarget = (np.array(dataset[targetClassName])).ravel()
    datsetReclassified[targetClassName] = [ originalClass if actualTarget[j] == pseudoClass
                                           else actualTarget[j]
                                           for j in range(len(actualTarget))]
    print(Counter(datsetReclassified[targetClassName]))
    return  datsetReclassified


print(Counter(X_train['percentage']))
newDS = pseudoClassCreation(X_train, 'disToRiv', 200, 2, 'percentage')
y = newDS['percentage']
newDS.drop(['percentage'], axis=1, inplace = True)
x_res,y_res = ms.randomUndersampling(newDS, y, )
x_res['percentage'] = y_res
# newDatase = revertPseudoClassCreation(x_res, 0, 2, 'percentage')


In [ ]:
x_res.to_csv('basin1ControlClass0Sampling4Class_ToSHP.csv',index = None)

# Data description and visualization

In [ ]:
#### import dataset to describe
DS= pd.read_csv('datasets/basin4_Training.csv', index_col=None)
DS.head()

In [ ]:
DS.drop(['x_coord','y_coord'], axis = 1, inplace=True)
DS.head()

In [ ]:
### FAcc vs Labels
targets = DS['percentage']
FAcc = original['FAcc']
FAcc_norm = DS['FAcc_norm']
fig, axs = plt.subplots(1, 2, figsize=(13,4), sharey=True)
fig.text(-0.02, 0.5, 'labels', va='center', rotation='vertical')
fig.text(0.5, 1, 'Flow accumulation vs labels distribution', ha ='center')
axs[0].scatter(FAcc,targets)
# axs[0].set_title("Facc")
axs[0].set(xlabel='a) Flow Accumulation')
axs[1].scatter(FAcc_norm,targets)
# axs[1].set_title("FAcc_norm")
axs[1].set(xlabel='b) Flow Accumulation estandardized')
plt.rcParams['font.size'] = '20'
fig.tight_layout()


In [ ]:

## Plot all features vs labels
# 'disToRiv', 'TWI', 'TPI', 'slope', 'elevation',

targets = DS['percentage']
# targets = np.where(targets == 5,2,targets)

E = DS['elevation'] 
slope = DS['slope']
FAcc = DS['FAcc']
TWI = DS['TWI']
TPI = DS['TPI']
DLSOL4R150 = DS['LDSOL4R150']
DLSOL5R150 = DS['LDSOL5R150']
DLSOL5R200 = DS['LDSOL5R200']
FAProx_01 = DS['FAProx_01']
FAProx_025 = DS['FAProx_025']
visibility = DS['visibility']

fig, axs = plt.subplots(4,3, figsize=(13, 8), sharey=True)
fig.supylabel('Labels')
plt.rcParams['font.size'] = '15'
plt.yticks([0,1,5])

'''
E = DS['elevation'] 
slope = DS['slope']
FAcc = DS['FAcc']
TWI = DS['TWI']
'''
axs[0, 0].scatter(E,targets)
axs[0, 0].set_title("Elevation")
axs[1, 0].scatter(slope,targets)
axs[1, 0].set_title("Slope")
axs[2, 0].scatter(FAcc,targets)
axs[2, 0].set_title("Flow accumulation")
axs[3, 0].scatter(TWI,targets)
axs[3, 0].set_title("TWI")

'''
TPI = DS['TPI']
DLSOL4R150 = DS['DLSOL4R150']
DLSOL5R150 = DS['DLSOL5R150']
DLSOL5R200 = DS['DLSOL5R200']
'''
axs[0, 1].scatter(TPI,targets)
axs[0, 1].set_title('TPI')
axs[1, 1].scatter(DLSOL4R150,targets)
axs[1, 1].set_title("DLSOL4R150")
axs[2, 1].scatter(DLSOL5R150,targets)
axs[2, 1].set_title("DLSOL5R150")
axs[3, 1].scatter(DLSOL5R200,targets)
axs[3, 1].set_title("DLSOL5R200")

'''
FAProx_01 = DS['FAProx_01']
FAProx_025 = DS['FAProx_025']
visibility = DS['visibility']
'''
axs[0, 2].scatter(FAProx_01,targets)
axs[0, 2].set_title('FAProx_01')
axs[1, 2].scatter(FAProx_025,targets)
axs[1, 2].set_title("FAProx_025")
axs[2, 2].scatter(visibility,targets)
axs[2, 2].set_title("Visibility")

fig.tight_layout()


In [ ]:
print(DS.head())
#  Return a dataset with the rows corresponding to the index where condition in DS.columName is valid. 
dsArray = DS[DS.percentage != 0] 
print(dsArray.head()) 

In [ ]:
sns.set(font_scale=1.5)
sns.pairplot(DS, hue = 'percentage', diag_kind = 'kde', 
             plot_kws = {'alpha': 0.8, 's': 100},
             height = 4, corner=True, palette = "Set2")# vars = ['life_exp', 'log_pop', 'log_gdp_per_cap'],

# sns.pairplot(DS, hue="percentage")

In [ ]:
####. Covariance Matrix
sns.set(font_scale=0.7)
matrix = DS.corr().round(2)
sns.heatmap(matrix, annot=True)
plt.set_figsize=(25,20)
plt.show()

In [ ]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier

estimator = RandomForestClassifier(criterion='entropy', random_state = 50)
x_train,y_train = ms.importDataSet('basin1Train.csv', 'percentage')
classifier = OneVsRestClassifier(estimator).fit(x_train,y_train)


In [ ]:
classifier = ms.loadModel('./outputs/2022-08-05/11-01-57/2208051101.pkl')
x_test,y_test = ms.importDataSet('basin1Test.csv', 'percentage')

x_test = ms.removeCoordinatesFromDataSet(x_test)

# y_prob = classifier.predict_proba(x_test)
#print(np.unique(y_prob))

In [ ]:
md.plot_ROC_AUC_OneVsRest(classifier, x_test, y_test)

In [ ]:
_,y_test = ms.importDataSet('./bestModels/Classifier/10-18-08/2208051018prediction_basin1Test.csv', 'prediction')
unique, count = np.unique(y_test, return_counts=True)
total = count.sum()
print(total)
percent = np.round(np.zeros_like(unique).astype('float16'),3)
print('values, counts , percent')
for i in range(len(unique)):    
   percent[i] = (count[i]/total)*100
   print(unique[i],"\t", count[i], percent[i])
